In [13]:
from random import choice
from tqdm.auto import tqdm
import numpy as np
from queue import PriorityQueue

In [14]:
PUZZLE_DIM = 3  
RANDOMIZE_STEPS = 100

def available_actions(state: np.ndarray) -> list:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(((x, y), (x - 1, y))) 
    if x < PUZZLE_DIM - 1:
        actions.append(((x, y), (x + 1, y)))  
    if y > 0:
        actions.append(((x, y), (x, y - 1))) 
    if y < PUZZLE_DIM - 1:
        actions.append(((x, y), (x, y + 1))) 
    return actions

def do_action(state: np.ndarray, action: tuple) -> np.ndarray:
    new_state = state.copy()
    (x1, y1), (x2, y2) = action
    new_state[x1, y1], new_state[x2, y2] = new_state[x2, y2], new_state[x1, y1]
    return new_state

def optimized_heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    manhattan_distance = 0
    misplacement_count = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                manhattan_distance += abs(x - goal_x) + abs(y - goal_y)
                if goal[x, y] != value:
                    misplacement_count += 1

    return manhattan_distance + 2 * misplacement_count  

def solve_puzzle(start_state: np.ndarray, goal_state: np.ndarray):
    frontier = PriorityQueue()
    frontier.put((0, start_state.tolist(), [start_state.tolist()], 0)) 
    visited = set()

    while not frontier.empty():
        f_cost, current_state, path, g_cost = frontier.get()
        current_state = np.array(current_state)
        if np.array_equal(current_state, goal_state):
            return path  

        state_tuple = tuple(map(tuple, current_state))
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_g_cost = g_cost + 1
            new_f_cost = new_g_cost + optimized_heuristic(new_state, goal_state)
            new_path = path + [new_state.tolist()]
            frontier.put((new_f_cost, new_state.tolist(), new_path, new_g_cost))

    return None  


In [15]:
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
   
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
solution = solve_puzzle(state, goal_state)

if solution:
    print(f"Soluzione trovata in {len(solution) - 1} moves!") 
    for step, s in enumerate(solution):
        print(f"Step {step}:")
        print(np.array(s))  
else:
    print("No solution found")

Randomizing: 100%|██████████| 100/100 [00:00<00:00, 53410.21it/s]

Soluzione trovata in 24 moves!
Step 0:
[[6 4 5]
 [2 8 3]
 [0 1 7]]
Step 1:
[[6 4 5]
 [0 8 3]
 [2 1 7]]
Step 2:
[[0 4 5]
 [6 8 3]
 [2 1 7]]
Step 3:
[[4 0 5]
 [6 8 3]
 [2 1 7]]
Step 4:
[[4 5 0]
 [6 8 3]
 [2 1 7]]
Step 5:
[[4 5 3]
 [6 8 0]
 [2 1 7]]
Step 6:
[[4 5 3]
 [6 0 8]
 [2 1 7]]
Step 7:
[[4 5 3]
 [0 6 8]
 [2 1 7]]
Step 8:
[[4 5 3]
 [2 6 8]
 [0 1 7]]
Step 9:
[[4 5 3]
 [2 6 8]
 [1 0 7]]
Step 10:
[[4 5 3]
 [2 6 8]
 [1 7 0]]
Step 11:
[[4 5 3]
 [2 6 0]
 [1 7 8]]
Step 12:
[[4 5 3]
 [2 0 6]
 [1 7 8]]
Step 13:
[[4 0 3]
 [2 5 6]
 [1 7 8]]
Step 14:
[[0 4 3]
 [2 5 6]
 [1 7 8]]
Step 15:
[[2 4 3]
 [0 5 6]
 [1 7 8]]
Step 16:
[[2 4 3]
 [1 5 6]
 [0 7 8]]
Step 17:
[[2 4 3]
 [1 5 6]
 [7 0 8]]
Step 18:
[[2 4 3]
 [1 0 6]
 [7 5 8]]
Step 19:
[[2 0 3]
 [1 4 6]
 [7 5 8]]
Step 20:
[[0 2 3]
 [1 4 6]
 [7 5 8]]
Step 21:
[[1 2 3]
 [0 4 6]
 [7 5 8]]
Step 22:
[[1 2 3]
 [4 0 6]
 [7 5 8]]
Step 23:
[[1 2 3]
 [4 5 6]
 [7 0 8]]
Step 24:
[[1 2 3]
 [4 5 6]
 [7 8 0]]
